In [169]:
import altair as alt
from altair_saver import save
import pydataset 
import itertools
import json
import random
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import os
import math
import toolz
import glob
import copy

## Render User Created Charts

In [124]:
FOLDER = 'UserC2'

In [62]:
scoreDF = pd.read_csv(os.path.join(FOLDER, 'scoreRecord.csv'))

In [128]:
UserBatch = pd.read_csv(os.path.join(FOLDER, 'HumanCreated_batch_results.csv'))

In [129]:
def getEditTimes(d):
    return len(json.loads(json.loads(d)[0]['history']))
editTimes = [getEditTimes(d) for d in UserBatch['Answer.taskAnswers'].values]

In [192]:
pd.DataFrame({'time': UserBatch['WorkTimeInSeconds'].values, 'edit': editTimes})\
    .to_csv(os.path.join(FOLDER, 'userc2HumanResult.csv'), index = False)

In [131]:
Height = 400
def render(_vSpec, aspectRatio, band, textR, orientation):
    if (_vSpec['encoding']['x']['type'] == 'ordinal'):
        _vSpecNum = _vSpec['encoding']['y']
        _vSpecCat = _vSpec['encoding']['x']
    else:
        _vSpecNum = _vSpec['encoding']['x']
        _vSpecCat = _vSpec['encoding']['y']        
         
    _vSpecCat['band'] = band
    _vSpecCat['axis']['labelAngle'] = textR 
   
    _vSpec['width'] = Height * aspectRatio
    _vSpec['height'] = Height
        
    if orientation == 0:
        _vSpec['encoding']['y'] = _vSpecNum
        _vSpec['encoding']['x'] = _vSpecCat
    else:       
        _vSpec['encoding']['x'] = _vSpecNum
        _vSpec['encoding']['y'] = _vSpecCat
        
    return _vSpec

In [155]:
idx = 0
for i, d in enumerate(UserBatch['Answer.taskAnswers'].values):
    vSpec = json.loads(json.loads(d)[0]['iSpec'])
    
    # set none => False
    if vSpec['encoding']['x']['type'] == 'quantitative':
        vSpec['encoding']['x']['axis']['grid'] = False
    else:
        vSpec['encoding']['y']['axis']['grid'] = False
    
    # Name is broken
    for j, x in enumerate(vSpec['data']['values']):
        x['Name'] = "".join(x['Name'])

    # parse fixed parameters
    NBar = len(vSpec['data']['values'])
    maxChar = max([len(x['Name']) for x in vSpec['data']['values']])
    
    # maxwidth
#     maxAspecRatio = min(800, vSpec['width'] + 100) / Height
    maxAspecRatio = int(json.loads(d)[0]['iMaxWidth']) / Height
    
    if NBar > 25:
        continue
        
    # render user 0
    chart = alt.Chart.from_dict(vSpec)
    namePrefix = str(idx) + '_' + '0'
    chart.save(os.path.join(FOLDER, 'User', namePrefix + '.png'))
    with open(os.path.join(FOLDER, 'User', namePrefix + '.json'), 'w') as f:
        json.dump(vSpec, f)
        
    # render opt 2
    print(idx, NBar, maxChar,maxAspecRatio )
    optPara = scoreDF[(scoreDF.nbar == NBar) & (scoreDF.char == maxChar) & (scoreDF.aspectRatio <= maxAspecRatio)].sort_values(by='score', ascending =False).iloc[0]
    optSpec = render(copy.deepcopy(vSpec), optPara['aspectRatio'], \
           optPara['bandwidth'], optPara['rotation'], optPara['orientation'])
    chart = alt.Chart.from_dict(optSpec)
    namePrefix = str(idx) + '_' + '2'
    chart.save(os.path.join(FOLDER, 'Opt', namePrefix + '.png'))
    with open(os.path.join(FOLDER, 'Opt', namePrefix + '.json'), 'w') as f:
        json.dump(optSpec, f)
        
    # render  3
    randSpec = render(copy.deepcopy(vSpec), random.sample(AspectRatioRanges, 1)[0], \
           random.sample(BandRanges, 1)[0], random.sample(TextRotationRanges, 1)[0], \
                     random.sample(OrientationRanges, 1)[0])
    chart = alt.Chart.from_dict(randSpec)
    namePrefix = str(idx) + '_' + '3'
    chart.save(os.path.join(FOLDER, 'Random', namePrefix + '.png'))
    with open(os.path.join(FOLDER, 'Random', namePrefix + '.json'), 'w') as f:
        json.dump(optSpec, f)
        
    # default 1
    dSpec = copy.deepcopy(vSpec)
    dSpec['width'] = maxAspecRatio * Height
    if dSpec['encoding']['x']['type'] == 'ordinal':
        dSpecCat = dSpec['encoding']['x']
        dSpecNum = dSpec['encoding']['y']
    else:
        dSpecCat = dSpec['encoding']['y']
        dSpecNum = dSpec['encoding']['x']
    del dSpecCat['band']
    del dSpecCat['axis']
    del dSpecCat['scale']['paddingInner']
    dSpec['encoding']['x'] = dSpecCat
    dSpec['encoding']['y'] = dSpecNum
    chart = alt.Chart.from_dict(dSpec)
    namePrefix = str(idx) + '_' + '1'
    chart.save(os.path.join(FOLDER, 'Default', namePrefix + '.png'))
    with open(os.path.join(FOLDER, 'Default', namePrefix + '.json'), 'w') as f:
        json.dump(optSpec, f)  
        
    idx = idx + 1

0 8 5 1.75
1 8 12 1.0
2 20 13 2.0
3 20 10 1.75
4 15 5 1.0
5 17 6 1.5
6 21 6 1.5
7 9 4 1.5
8 17 6 1.5
9 24 9 2.0
10 20 6 0.75
11 16 6 1.75
12 20 4 2.0
13 13 13 2.0
14 8 7 1.5
15 20 8 1.25
16 7 8 2.0
17 5 6 2.0
18 21 9 2.0
19 24 6 1.25
20 20 12 1.0
21 12 3 1.0
22 5 6 1.25
23 15 6 1.75
24 17 11 2.0
25 19 6 1.75
26 9 3 1.75
27 5 7 2.0
28 13 11 0.75
29 15 11 1.0
30 21 9 0.75
31 13 11 1.5
32 14 4 1.75
33 15 8 2.0
34 9 5 0.75
35 16 6 1.25
36 19 10 1.25
37 19 7 1.75
38 6 11 1.0
39 20 13 1.5
40 10 13 1.5
41 21 12 1.75
42 12 7 1.0
43 18 13 2.0
44 16 10 0.75
45 8 4 0.75
46 22 11 0.75
47 6 6 1.25
48 24 4 1.75
49 5 10 1.25
50 17 11 1.75
51 12 3 1.5
52 11 9 2.0
53 11 8 1.0
54 19 12 2.0
55 22 11 1.25
56 17 4 1.75
57 13 7 1.75
58 11 7 1.0
59 23 10 1.75
60 23 9 2.0
61 22 10 2.0
62 10 8 1.75
63 10 6 1.25
64 24 11 1.25
65 23 8 2.0
66 24 8 0.75
67 22 12 1.5
68 8 13 0.75
69 24 9 1.25
70 11 3 1.25
71 8 5 1.5
72 19 10 1.25
73 17 10 1.0
74 14 9 1.5
75 23 7 1.75


In [43]:
json.loads(json.loads(d)[0]['data'])

[{'Name': ['chevrolet chevelle malibu'], 'Horsepower': 130},
 {'Name': ['buick skylark 32', ''], 'Horsepower': 165},
 {'Name': ['plymouth satellite'], 'Horsepower': 150},
 {'Name': ['amc rebel sst'], 'Horsepower': 150},
 {'Name': ['ford torino'], 'Horsepower': 140},
 {'Name': ['ford galaxie 5', '', ''], 'Horsepower': 198},
 {'Name': ['chevrolet impala'], 'Horsepower': 220}]

In [156]:
folder = 'UserC2'
def getFilename(path):
    return "https://xxxxxxxxxxxx/u2/" + os.path.split(path)[1]
humanC = [getFilename(x) for x in glob.glob(os.path.join(folder, "User", "*.png"))]
optC = [getFilename(x) for x in glob.glob(os.path.join(folder, "Opt", "*.png"))]
defaultC = [getFilename(x) for x in glob.glob(os.path.join(folder, "Default", "*.png"))]
randomC = [getFilename(x) for x in glob.glob(os.path.join(folder, "Random", "*.png"))]

humanC.sort()
optC.sort()
defaultC.sort()
randomC.sort()

fourwayZip = list(zip(humanC, optC, defaultC, randomC))
pairwiseCompare = [list(itertools.combinations(cc, 2)) for cc in fourwayZip]
pairs = list(itertools.chain(*pairwiseCompare))

np.random.shuffle(pairs)
def zipForCompare(pairs, perBatch = 9):
    if len(pairs) % perBatch != 0:
        remaining = perBatch - len(pairs) % perBatch
        pairs.extend(pairs[:remaining])
    
    zipPairs = [list(itertools.chain(*pairs[i:i+perBatch])) for i in range(0, len(pairs), perBatch)]
    columnNames = []
    for i in range(1, perBatch + 1):
        columnNames.append('img' + str(i) + '_1')
        columnNames.append('img' + str(i) + '_2')
    return pd.DataFrame(zipPairs, columns=columnNames)
#     zipPairsDF.head()   
#     zipPairsDF.to_csv('exp7.csv', index = False)
zipForCompare(pairs).to_csv(os.path.join(folder, 'aturk.csv'), index = False)

In [183]:
pairCount = resultPair[(resultPair.img1Gp == "1") & (resultPair.img2Gp == "3")].groupby(['cIdx', 'wIdx']).size().unstack().fillna(0)
# pairCount.head()
wilcoxon(pairCount[1].values, pairCount[2].values)

WilcoxonResult(statistic=1023.5, pvalue=0.2688451314616589)